In [1]:
import pandas as pd

In [2]:
#%% Load your main dataset
df = pd.read_csv("data/feature_engineering_train_delivery1.csv", index_col=0)

# Load the FIPS reference file
county_fips = pd.read_csv("data/county_fips_master.csv", encoding="latin1")

In [3]:
county_fips["county_name"] = county_fips["county_name"].str.replace(" County", "", case=False).str.strip().str.upper()

# Inspect the FIPS reference file to confirm column names
print(county_fips.head())

   fips county_name state_abbr state_name          long_name  sumlev  region  \
0  1001     AUTAUGA         AL    Alabama  Autauga County AL    50.0     3.0   
1  1003     BALDWIN         AL    Alabama  Baldwin County AL    50.0     3.0   
2  1005     BARBOUR         AL    Alabama  Barbour County AL    50.0     3.0   
3  1007        BIBB         AL    Alabama     Bibb County AL    50.0     3.0   
4  1009      BLOUNT         AL    Alabama   Blount County AL    50.0     3.0   

   division  state  county crosswalk region_name       division_name  
0       6.0    1.0     1.0   3-6-1-1       South  East South Central  
1       6.0    1.0     3.0   3-6-1-3       South  East South Central  
2       6.0    1.0     5.0   3-6-1-5       South  East South Central  
3       6.0    1.0     7.0   3-6-1-7       South  East South Central  
4       6.0    1.0     9.0   3-6-1-9       South  East South Central  


In [4]:
# Merge the main dataset with the FIPS file
# Assuming `County` in `county_fips` matches `County of Injury` in `df`
#This is the merge based on the county name
df = df.merge(county_fips, left_on="County of Injury", right_on="county_name", how="left")

# Ensure FIPS codes are zero-padded to 5 digits
df["fips"] = df["fips"].astype(str).str.zfill(5)

In [5]:

# Verify the merged data
print("Merged DataFrame:")
df.head()

Merged DataFrame:


,Age at Injury,Alternative Dispute Resolution,Attorney/Representative,Average Weekly Wage,Birth Year,C-3 Date,Carrier Name,Carrier Type,County of Injury,COVID-19 Indicator,...,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,50.0,0.0,1,1500.0,1971.0,1,STATE INSURANCE FUND,2A. SIF,RICHMOND,0,...,Georgia,Richmond County GA,50.0,3.0,5.0,13.0,245.0,3-5-13-245,South,South Atlantic
1,50.0,0.0,1,1500.0,1971.0,1,STATE INSURANCE FUND,2A. SIF,RICHMOND,0,...,New York,Richmond County NY,50.0,1.0,2.0,36.0,85.0,1-2-36-85,Northeast,Middle Atlantic
2,50.0,0.0,1,1500.0,1971.0,1,STATE INSURANCE FUND,2A. SIF,RICHMOND,0,...,North Carolina,Richmond County NC,50.0,3.0,5.0,37.0,153.0,3-5-37-153,South,South Atlantic
3,50.0,0.0,1,1500.0,1971.0,1,STATE INSURANCE FUND,2A. SIF,RICHMOND,0,...,Virginia,Richmond County VA,50.0,3.0,5.0,51.0,159.0,3-5-51-159,South,South Atlantic
4,45.0,0.0,0,250.0,1976.0,0,STATE INSURANCE FUND,2A. SIF,NASSAU,0,...,Florida,Nassau County FL,50.0,3.0,5.0,12.0,89.0,3-5-12-89,South,South Atlantic


In [6]:
test = df.groupby("fips").size().reset_index(name="accident_count")

In [7]:
test.head()

,fips,accident_count
0,00nan,872
1,1059.0,1111
2,1063.0,1379
3,1073.0,2513
4,1089.0,1881


In [8]:
df.columns

Index(['Age at Injury', 'Alternative Dispute Resolution',
       'Attorney/Representative', 'Average Weekly Wage', 'Birth Year',
       'C-3 Date', 'Carrier Name', 'Carrier Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'WCIO Cause of Injury Code',
       'WCIO Cause of Injury Description', 'WCIO Nature of Injury Code',
       'WCIO Nature of Injury Description', 'WCIO Part Of Body Code',
       'WCIO Part Of Body Description', 'Zip Code', 'Number of Dependents',
       'Assembly Year', 'Accident Year', 'C-2 Date Year', 'Assembly Month',
       'Accident Month', 'C-2 Date Month', 'Age Group',
       'Frequent Injury Cause', 'Broad Body Part',
       'Dependency-to-Income Ratio', 'Injury-Location Pair',
       'Time Between Events', 'Accident on Weekday', 'Injury Complexity',
       'Carrier Accident Density', 'Season of Accident',
  

In [9]:
# Check the value counts of the fips column
print(df['fips'].value_counts().sort_index())

00nan        872
1059.0      1111
1063.0      1379
1073.0      2513
1089.0      1881
           ...  
56001.0    10476
6031.0     41093
6059.0     13552
8059.0      2513
8121.0      1410
Name: fips, Length: 324, dtype: int64


In [10]:
#save the indexes where the fips is '00nan'
nan_indexes = df[df['fips'] == '00nan'].index


#drop the rows where the fips is '00nan'
df = df.drop(nan_indexes)



In [11]:
df['fips'].value_counts().sort_index()

1059.0      1111
1063.0      1379
1073.0      2513
1089.0      1881
1099.0     18819
           ...  
56001.0    10476
6031.0     41093
6059.0     13552
8059.0      2513
8121.0      1410
Name: fips, Length: 323, dtype: int64

In [12]:
# Save the merged DataFrame if needed
df.to_csv("data/merged_with_fips.csv")